## Special Effects

In [2]:
import numpy as np
import cv2
import pafy

### Code to capture video

In [8]:
cap = cv2.VideoCapture(file) 
   
# Check if camera opened successfully 
if (cap.isOpened()== False):  
  print("Error opening video  file") 
   
# Read until video is completed 
while(cap.isOpened()): 
      
  # Capture frame-by-frame 
  ret, frame = cap.read() 
  if ret == True: 
   
    # Display the resulting frame 
    cv2.imshow('Frame', frame) 
    
    # Press Q on keyboard to  exit 
    if cv2.waitKey(25) & 0xFF == ord('q'): 
      break
   
  # Break the loop 
  else:  
    break
   
# # When everything done, release  
# # the video capture object 
# cap.release() 
   
# # Closes all the frames 
# cv2.destroyAllWindows() 

Error opening video  file


### Code to capture video with error handling

In [3]:
import cv2

cap = cv2.VideoCapture(file)
while not cap.isOpened():
    cap = cv2.VideoCapture(file)
    if cv2.waitKey(1000) & 0xFF == ord('q'): 
          break
    print("Wait for the header")

pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
while True:
    flag, frame = cap.read()
    if flag:
        # The frame is ready and already captured
        cv2.imshow('video', frame)
        
        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print(str(pos_frame)+" frames")
        
        # Press Q on keyboard to  exit 
        if cv2.waitKey(25) & 0xFF == ord('q'): 
          break
        
    else:
        # The next frame is not ready, so we try to read it again
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
        print("frame is not ready")
        # It is better to wait for a while for the next frame to be ready
        if cv2.waitKey(1000) & 0xFF == ord('q'): 
          break

    if cv2.waitKey(10) == 27:
        break
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        # If the number of captured frames is equal to the total number of frames,
        # we stop
        break
        
# When everything done, release  
# the video capture object 
cap.release() 
   
# Closes all the frames 
cv2.destroyAllWindows() 

1.0 frames
2.0 frames
3.0 frames
4.0 frames
5.0 frames
6.0 frames
7.0 frames
8.0 frames
9.0 frames
10.0 frames
11.0 frames
12.0 frames
13.0 frames
14.0 frames
15.0 frames
16.0 frames
17.0 frames
18.0 frames
19.0 frames
20.0 frames
21.0 frames
22.0 frames
23.0 frames
24.0 frames
25.0 frames
26.0 frames
27.0 frames
28.0 frames
29.0 frames
30.0 frames
31.0 frames
32.0 frames
33.0 frames
34.0 frames
35.0 frames
36.0 frames
37.0 frames
38.0 frames
39.0 frames
40.0 frames
41.0 frames
42.0 frames
43.0 frames
44.0 frames
45.0 frames
46.0 frames
47.0 frames
48.0 frames
49.0 frames
50.0 frames
51.0 frames
52.0 frames
53.0 frames


### Run and Save youtube video

In [3]:
def getURL(url):
    vPafy = pafy.new(url)
    play = vPafy.getbest(preftype='mp4')
    return play.url

def captureVideo(url):
    cap = cv2.VideoCapture(url)
    while not cap.isOpened():
        cap = cv2.VideoCapture(url)
        if cv2.waitKey(1000) & 0xFF == ord('q'): 
              break
        print("Wait for the header")
    # Check if camera opened successfully 
    if (cap.isOpened()== False):  
      print("Error opening video  file") 
    return cap.isOpened(), cap

def getFrame(cap, pos_frame):
    flag, frame = cap.read()
    while not flag:      
        # The next frame is not ready, so we try to read it again
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
        print("frame is not ready")
        # It is better to wait for a while for the next frame to be ready
        if cv2.waitKey(1000) & 0xFF == ord('q'): 
          break
        flag, frame = cap.read()
    return flag, frame

def save(url, filename='output'):
    url = getURL(url)
    ret, cap = captureVideo(url)
    if not ret:
        print('Unable to save')
        return
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(filename+'.mp4', fourcc, cap.get(5), (int(cap.get(3)),int(cap.get(4))))

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            out.write(frame)

            cv2.imshow('frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()    

### Rotation, Scaling effects with weighted average

In [4]:
def run(file, alpha=0.8, effect=False, record=False):
    ret, cap = captureVideo(file)
    if not ret:
        print('Unable to capture video')
        return
    
    if record:
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        out = cv2.VideoWriter('1'+'.mp4', fourcc, 5, (int(2*cap.get(3)),int(cap.get(4))))
    
    width = int(cap.get(3))
    height = int(cap.get(4))
    scale = 1.0
    scale_inc = 0.02
    rot = 0
    rot_inc = 0.01
    
    pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
    ret, frame = getFrame(cap, pos_frame)
    while ret:
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames, we stop
            break
            
        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        ret1, frame1 = getFrame(cap, pos_frame)
        
        if not ret1:
            break
        
        if effect:
            frame = np.clip(alpha*frame.astype('int')+(1-alpha)*frame1.astype('int'), 0, 255).astype('uint8')
            M = cv2.getRotationMatrix2D((width/2, height/2), rot, scale)
            if scale + scale_inc > 2:
                scale = 2
                scale_inc = -scale_inc
            elif scale + scale_inc < 1:
                scale = 1
                scale_inc = -scale_inc
            else:
                scale += scale_inc

            if rot + rot_inc > 360:
                rot = 180
                rot_inc = -rot_inc
            elif rot + rot_inc < 0:
                rot = 0
                rot_inc = -rot_inc
            else:
                rot += rot_inc  
            frame = cv2.warpAffine(frame, M, (width, height))
        else:
            frame = frame1
        
        cv2.imshow('video', frame)
        
        print(str(pos_frame)+" frames")
        
        if record and int(pos_frame) >= 480 and int(pos_frame) <= 495:
            out.write(np.hstack([frame1, frame]))
        
#         if int(pos_frame) == 485:
#             cv2.imwrite('tmp.png', np.hstack([frame1, frame]))
#             break
            
        # Press Q on keyboard to  exit 
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        
    # When everything done, release  
    # the video capture object 
    cap.release() 
    if record:
        out.release()
    # Closes all the frames 
    cv2.destroyAllWindows()         

In [6]:
# url = 'https://youtu.be/ZSt9tm3RoUU'
# save(url, 'our_story')

# url = 'https://youtu.be/rMPkUuMq024'
# save(url, 'seen')

1.0 frames
2.0 frames
3.0 frames
4.0 frames
5.0 frames
6.0 frames
7.0 frames
8.0 frames
9.0 frames
10.0 frames
11.0 frames
12.0 frames
13.0 frames
14.0 frames
15.0 frames
16.0 frames
17.0 frames
18.0 frames
19.0 frames
20.0 frames
21.0 frames
22.0 frames
23.0 frames
24.0 frames
25.0 frames
26.0 frames
27.0 frames
28.0 frames
29.0 frames
30.0 frames
31.0 frames
32.0 frames
33.0 frames
34.0 frames
35.0 frames
36.0 frames
37.0 frames
38.0 frames
39.0 frames
40.0 frames
41.0 frames
42.0 frames
43.0 frames
44.0 frames
45.0 frames
46.0 frames
47.0 frames
48.0 frames
49.0 frames
50.0 frames
51.0 frames
52.0 frames
53.0 frames
54.0 frames
55.0 frames
56.0 frames
57.0 frames
58.0 frames
59.0 frames
60.0 frames
61.0 frames
62.0 frames
63.0 frames
64.0 frames
65.0 frames
66.0 frames
67.0 frames
68.0 frames
69.0 frames
70.0 frames
71.0 frames
72.0 frames
73.0 frames
74.0 frames
75.0 frames
76.0 frames
77.0 frames
78.0 frames
79.0 frames
80.0 frames
81.0 frames
82.0 frames
83.0 frames
84.0 frames
8

In [11]:
file = 'our_story.mp4'
run(file, alpha = 0.6, effect=True, record=True)

1.0 frames
2.0 frames
3.0 frames
4.0 frames
5.0 frames
6.0 frames
7.0 frames
8.0 frames
9.0 frames
10.0 frames
11.0 frames
12.0 frames
13.0 frames
14.0 frames
15.0 frames
16.0 frames
17.0 frames
18.0 frames
19.0 frames
20.0 frames
21.0 frames
22.0 frames
23.0 frames
24.0 frames
25.0 frames
26.0 frames
27.0 frames
28.0 frames
29.0 frames
30.0 frames
31.0 frames
32.0 frames
33.0 frames
34.0 frames
35.0 frames
36.0 frames
37.0 frames
38.0 frames
39.0 frames
40.0 frames
41.0 frames
42.0 frames
43.0 frames
44.0 frames
45.0 frames
46.0 frames
47.0 frames
48.0 frames
49.0 frames
50.0 frames
51.0 frames
52.0 frames
53.0 frames
54.0 frames
55.0 frames
56.0 frames
57.0 frames
58.0 frames
59.0 frames
60.0 frames
61.0 frames
62.0 frames
63.0 frames
64.0 frames
65.0 frames
66.0 frames
67.0 frames
68.0 frames
69.0 frames
70.0 frames
71.0 frames
72.0 frames
73.0 frames
74.0 frames
75.0 frames
76.0 frames
77.0 frames
78.0 frames
79.0 frames
80.0 frames
81.0 frames
82.0 frames
83.0 frames
84.0 frames
8

643.0 frames
644.0 frames
645.0 frames
646.0 frames
647.0 frames
648.0 frames
649.0 frames
650.0 frames
651.0 frames
652.0 frames
653.0 frames
654.0 frames
655.0 frames
656.0 frames
657.0 frames
658.0 frames
659.0 frames
660.0 frames
661.0 frames
662.0 frames
663.0 frames
664.0 frames
665.0 frames
666.0 frames
667.0 frames
668.0 frames
669.0 frames
670.0 frames
671.0 frames


### Combine two videos weighted by alpha

In [7]:
def run1(fC, fS, alpha=0.8, record=False):
    retC, capC = captureVideo(fC)
    retS, capS = captureVideo(fS)
    height = int(min(capC.get(4), capS.get(4)))
    
    if record:
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        out = cv2.VideoWriter('2'+'.mp4', fourcc, 5, (int(3*capC.get(3)),height))
    
    if not retC or not retS:
        print('Unable to capture video')
        return
        
    while True:
        pos_frame = capC.get(cv2.CAP_PROP_POS_FRAMES)
        retC, frameC = getFrame(capC, pos_frame)
        retS, frameS = getFrame(capS, pos_frame)
        
        if not retC or not retS:
            break
        
        frame = np.clip(alpha*frameC[:height, :].astype('int')+(1-alpha)*frameS[:height, :].astype('int'), 0, 255).astype('uint8')
        cv2.imshow('video', frame)
        print(str(pos_frame)+" frames")
        
        if record and int(pos_frame) >= 482 and int(pos_frame) <= 497:
            out.write(np.hstack([frameC[:height,:], frameS[:height,:], frame]))
        
#         if int(pos_frame) == 485:
#             cv2.imwrite('tmp.png', np.hstack([frameC[:height,:], frameS[:height,:], frame]))
#             break
        
        if capC.get(cv2.CAP_PROP_POS_FRAMES) == capC.get(cv2.CAP_PROP_FRAME_COUNT) or capS.get(cv2.CAP_PROP_POS_FRAMES) == capS.get(cv2.CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames, we stop
            break
        
        # Press Q on keyboard to  exit 
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        
    # When everything done, release  
    # the video capture object 
    capC.release()
    capS.release()
    if record:
        out.release()
    # Closes all the frames 
    cv2.destroyAllWindows()         

In [8]:
fS = 'our_story.mp4'
fC = 'seen.mp4'
run1(fC, fS, alpha = 0.7, record=True)

0.0 frames
1.0 frames
2.0 frames
3.0 frames
4.0 frames
5.0 frames
6.0 frames
7.0 frames
8.0 frames
9.0 frames
10.0 frames
11.0 frames
12.0 frames
13.0 frames
14.0 frames
15.0 frames
16.0 frames
17.0 frames
18.0 frames
19.0 frames
20.0 frames
21.0 frames
22.0 frames
23.0 frames
24.0 frames
25.0 frames
26.0 frames
27.0 frames
28.0 frames
29.0 frames
30.0 frames
31.0 frames
32.0 frames
33.0 frames
34.0 frames
35.0 frames
36.0 frames
37.0 frames
38.0 frames
39.0 frames
40.0 frames
41.0 frames
42.0 frames
43.0 frames
44.0 frames
45.0 frames
46.0 frames
47.0 frames
48.0 frames
49.0 frames
50.0 frames
51.0 frames
52.0 frames
53.0 frames
54.0 frames
55.0 frames
56.0 frames
57.0 frames
58.0 frames
59.0 frames
60.0 frames
61.0 frames
62.0 frames
63.0 frames
64.0 frames
65.0 frames
66.0 frames
67.0 frames
68.0 frames
69.0 frames
70.0 frames
71.0 frames
72.0 frames
73.0 frames
74.0 frames
75.0 frames
76.0 frames
77.0 frames
78.0 frames
79.0 frames
80.0 frames
81.0 frames
82.0 frames
83.0 frames
84

639.0 frames
640.0 frames
641.0 frames
642.0 frames
643.0 frames
644.0 frames
645.0 frames
646.0 frames
647.0 frames
648.0 frames
649.0 frames
650.0 frames
651.0 frames
652.0 frames
653.0 frames
654.0 frames
655.0 frames
656.0 frames
657.0 frames
658.0 frames
659.0 frames
660.0 frames
661.0 frames
662.0 frames
663.0 frames
664.0 frames
665.0 frames
666.0 frames
667.0 frames
668.0 frames
669.0 frames
670.0 frames
671.0 frames
672.0 frames
673.0 frames
674.0 frames
675.0 frames
676.0 frames
677.0 frames
678.0 frames
679.0 frames
680.0 frames
681.0 frames
682.0 frames
683.0 frames
684.0 frames
685.0 frames
686.0 frames
687.0 frames
688.0 frames
689.0 frames
690.0 frames
691.0 frames
692.0 frames
693.0 frames
694.0 frames
695.0 frames
696.0 frames
697.0 frames
698.0 frames
699.0 frames
700.0 frames
701.0 frames
702.0 frames
703.0 frames
704.0 frames
705.0 frames
706.0 frames
707.0 frames
708.0 frames
709.0 frames
710.0 frames
711.0 frames
712.0 frames
713.0 frames
714.0 frames
715.0 frames

1250.0 frames
1251.0 frames
1252.0 frames
1253.0 frames
1254.0 frames
1255.0 frames
1256.0 frames
1257.0 frames
1258.0 frames
1259.0 frames
1260.0 frames
1261.0 frames
1262.0 frames
1263.0 frames
1264.0 frames
1265.0 frames
1266.0 frames
1267.0 frames
1268.0 frames
1269.0 frames
1270.0 frames
1271.0 frames
1272.0 frames
1273.0 frames
1274.0 frames
1275.0 frames
1276.0 frames
1277.0 frames
1278.0 frames
1279.0 frames
1280.0 frames
1281.0 frames
1282.0 frames
1283.0 frames
1284.0 frames
1285.0 frames
1286.0 frames
1287.0 frames
1288.0 frames
1289.0 frames
1290.0 frames
1291.0 frames


In [6]:
import tensorflow as tf

In [8]:
fS = 'our_story.mp4'
fC = 'seen.mp4'

capC = cv2.VideoCapture(fC)
capS = cv2.VideoCapture(fS)
W = tf.Variable(np.zeros((int(min(capC.get(7), capS.get(7)))), dtype=float))
    
def Cost():
    fS = 'our_story.mp4'
    fC = 'seen.mp4'
    
    capC = cv2.VideoCapture(fC)
    capS = cv2.VideoCapture(fS)
    height = int(min(capC.get(4), capS.get(4)))

    alpha = 0.6
    beta = 0.4
#     W = tf.Variable(np.zeros((int(min(capC.get(7), capS.get(7)))), dtype=float))
    Ws = tf.sigmoid(W)

    cost = 0

    while True:
        pos_frame = int(capC.get(cv2.CAP_PROP_POS_FRAMES))
        _, frameC = capC.read()
        _, frameS = capS.read()

        frameC = frameC[:height, :].ravel() / 255
        frameS = frameS[:height, :].ravel() / 255

        frame = Ws[pos_frame] * frameC + (1 - Ws[pos_frame]) * frameS

        JC = tf.reduce_mean(tf.square(tf.subtract(frame, frameC)))
        JS = tf.reduce_mean(tf.square(tf.subtract(frame, frameS)))

        cost += alpha * JC + beta * JS

        if capC.get(cv2.CAP_PROP_POS_FRAMES) == capC.get(cv2.CAP_PROP_FRAME_COUNT) or capS.get(cv2.CAP_PROP_POS_FRAMES) == capS.get(cv2.CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames, we stop
            break

        # Press Q on keyboard to  exit 
        if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

    # When everything done, release  
    # the video capture object 
    capC.release()
    capS.release()
    # Closes all the frames 
    cv2.destroyAllWindows()
    
    return cost

In [ ]:
# optimizer = tf.optimizers.Adam(learning_rate=0.01).minimize(Cost, var_list=[W])